# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [35]:
# your code here
import pandas as pd
df = pd.read_csv("/Users/ayeesha/ironhack/lab-nlp/your-code/Sentiment140.csv")
df = df.sample(20000)
df = df.reset_index()

print(df.shape)
df.head()

(20000, 7)


,index,target,ids,date,flag,user,text
0,322713,0,2004544608,Tue Jun 02 08:21:59 PDT 2009,NO_QUERY,Dominicanflower,"@YoungQ i feel left out again, cant chat live ..."
1,721775,0,2261292572,Sat Jun 20 20:33:50 PDT 2009,NO_QUERY,mel_says,@baileeann I have a feeling I won't get to go ...
2,1480575,4,2066884826,Sun Jun 07 11:21:04 PDT 2009,NO_QUERY,MissDCember,"@erdnuckel86: Whiskey??? Urgh!! By the way, my..."
3,827375,4,1556844588,Sun Apr 19 00:37:44 PDT 2009,NO_QUERY,ryanz0r,Off to the 22nd TTA Team Championships to watc...
4,177184,0,1965555022,Fri May 29 16:07:35 PDT 2009,NO_QUERY,rickdog,tMeme&gt; My besties. If only @ddlovato was th...


In [36]:
import re
# string = """@Ironhack's-#Q website 776-is http://ironhack.com [(2018)]")"""
def clean_up(tweet):
    
    url_pattern = 'http\S+|www\S+'
    spec_chars_num_pattern = re.compile('[^a-zA-Z\s]') #means any character that IS NOT a-z OR A-Z
    space_pattern = re.compile(' +') # removing the double space
    tweet = tweet.lower()  
    tweet = re.sub(url_pattern, '', tweet)
    # Remove special chars and numbers
    tweet = re.sub(spec_chars_num_pattern,' ',tweet)
    tweet = re.sub(space_pattern, ' ', tweet)
    return tweet

from nltk.tokenize import word_tokenize
import nltk

def tokenize(tweet):
    tokens = word_tokenize(tweet)
    return tokens
    
from nltk.stem import PorterStemmer
nltk.download('wordnet') # wordnet is the most well known lemmatizer for english
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('omw-1.4')

def stem_and_lemmatize(tokens):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in tokens]
    lemmatizer = WordNetLemmatizer() 
    return[lemmatizer.lemmatize(word) for word in stemmed]

from nltk.corpus import stopwords
def remove_stopwords(tokens):
    without_sw = [word for word in tokens if not word in stopwords.words('english')]
    return without_sw

[nltk_data] Downloading package wordnet to /Users/ayeesha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ayeesha/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [40]:
def all_functions(tweet):

    cleaned_string = clean_up(tweet)
    tokenized_sentence = tokenize(cleaned_string)
    stem_lemmatize_lst = stem_and_lemmatize(tokenized_sentence)
    remove_stop_words = remove_stopwords(stem_lemmatize_lst)
    return remove_stop_words


In [41]:
df['text_processed'] = df['text'].apply(all_functions) 

In [42]:
df.head()

,index,target,ids,date,flag,user,text,text_processed
0,322713,0,2004544608,Tue Jun 02 08:21:59 PDT 2009,NO_QUERY,Dominicanflower,"@YoungQ i feel left out again, cant chat live ...","[youngq, feel, left, cant, chat, live, work, t..."
1,721775,0,2261292572,Sat Jun 20 20:33:50 PDT 2009,NO_QUERY,mel_says,@baileeann I have a feeling I won't get to go ...,"[baileeann, feel, get, go, either]"
2,1480575,4,2066884826,Sun Jun 07 11:21:04 PDT 2009,NO_QUERY,MissDCember,"@erdnuckel86: Whiskey??? Urgh!! By the way, my...","[erdnuckel, whiskey, urgh, way, myspac, messag..."
3,827375,4,1556844588,Sun Apr 19 00:37:44 PDT 2009,NO_QUERY,ryanz0r,Off to the 22nd TTA Team Championships to watc...,"[nd, tta, team, championship, watch, megan, ma..."
4,177184,0,1965555022,Fri May 29 16:07:35 PDT 2009,NO_QUERY,rickdog,tMeme&gt; My besties. If only @ddlovato was th...,"[tmeme, gt, besti, onli, ddlovato, wa, twitpic..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [57]:
# your code here
import nltk
from nltk.probability import FreqDist

regular_list = df['text_processed'].values
#https://stackabuse.com/python-how-to-flatten-list-of-lists/
flat_list = [item for sublist in regular_list for item in sublist]
fdist=FreqDist(flat_list)
top_words = list(fdist.keys())[:5000]
top_words

['youngq',
 'feel',
 'left',
 'cant',
 'chat',
 'live',
 'work',
 'thi',
 'suck',
 'baileeann',
 'get',
 'go',
 'either',
 'erdnuckel',
 'whiskey',
 'urgh',
 'way',
 'myspac',
 'messag',
 'sent',
 'nd',
 'tta',
 'team',
 'championship',
 'watch',
 'megan',
 'mark',
 'kick',
 'tmeme',
 'gt',
 'besti',
 'onli',
 'ddlovato',
 'wa',
 'twitpic',
 'sad',
 'aussi',
 'fan',
 'know',
 'guy',
 'would',
 'make',
 'could',
 'loyer',
 'jealou',
 'dumb',
 'homework',
 'real',
 'fun',
 'ride',
 'day',
 'wna',
 'show',
 'babi',
 'acema',
 'veri',
 'good',
 'nice',
 'plan',
 'home',
 'gah',
 'delay',
 'much',
 'possibl',
 'cours',
 'rob',
 'rep',
 'lmao',
 'yup',
 'everybodi',
 'got',
 'breezi',
 'greali',
 'right',
 'margaret',
 'gut',
 'next',
 'year',
 'seri',
 'though',
 'garveyben',
 'back',
 'girl',
 'see',
 'hous',
 'leav',
 'saturday',
 'queenofdesir',
 'yep',
 'probabl',
 'spot',
 'stress',
 'worst',
 'thing',
 'ever',
 'wait',
 'friday',
 'freedom',
 'abl',
 'face',
 'mess',
 'silliman',
 'wh

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [58]:
# your code here
#word = w
def find_features(tweet):
    words = set(tweet) # removing duplicates
    features = {}
    for w in top_words:
        features[w] = (w in words)
    return features


In [71]:
# tweets = df['text_processed'].values
# sentiments = df['target'].values

# features = []
# for tweet in tweets:
#     features.append(find_features(tweet))
# features[:2]
features = [(find_features(row['text_processed']), row['target'])  for _, row in df.iterrows()]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [72]:
# your code here

featuresets=features
train_set =featuresets[:2500]
test_set =featuresets[2500:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

classifier.show_most_informative_features()
#print('Naive Bayes accuracy: ',nltk.classify.accuracy(classifier, test))

Most Informative Features
                    suck = True                0 : 4      =      9.5 : 1.0
                     wow = True                4 : 0      =      8.9 : 1.0
                    hurt = True                0 : 4      =      8.3 : 1.0
                 without = True                0 : 4      =      7.7 : 1.0
                  beauti = True                4 : 0      =      7.6 : 1.0
                    hate = True                0 : 4      =      7.5 : 1.0
                    fail = True                0 : 4      =      7.1 : 1.0
                    woke = True                0 : 4      =      7.1 : 1.0
                       k = True                4 : 0      =      6.9 : 1.0
                   great = True                4 : 0      =      6.8 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [75]:
# your code here
classifier_test = nltk.classify.accuracy(classifier, test_set)
classifier.show_most_informative_features()
print(classifier_test)

Most Informative Features
                    suck = True                0 : 4      =      9.5 : 1.0
                     wow = True                4 : 0      =      8.9 : 1.0
                    hurt = True                0 : 4      =      8.3 : 1.0
                 without = True                0 : 4      =      7.7 : 1.0
                  beauti = True                4 : 0      =      7.6 : 1.0
                    hate = True                0 : 4      =      7.5 : 1.0
                    fail = True                0 : 4      =      7.1 : 1.0
                    woke = True                0 : 4      =      7.1 : 1.0
                       k = True                4 : 0      =      6.9 : 1.0
                   great = True                4 : 0      =      6.8 : 1.0
0.6814285714285714


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here